In [ ]:
import os
import cv2
import time

from ultralytics import YOLO


In [ ]:
# Загрузка предобученной модели YOLOv10 (можно 'yolov10n.pt', 'yolov10s.pt' или yolov10m.pt)
# model = YOLO('yolov10n.pt')  # nano-версия (быстрая, но менее точная)

model = YOLO('yolov10m.pt')  # рекомендуемая модель к обучению

# Обучение модели
results = model.train(data='dataset/data.yaml', epochs=100, imgsz=640, model='yolov8=10m.pt')

# подумать с моделью!

In [ ]:
model = YOLO('yolov8s-seg.pt')  # или yolov8n-seg.pt для быстрого старта
model.train(data='data.yaml', epochs=100, imgsz=640, batch=16,
            augment=True, lr0=0.01, patience=50, project='runs/exp_passengers')


In [ ]:
# Загружаем предобученную модель YOLOv10m
model = YOLO('yolov10m.pt')  # уже обучена на COCO

# Тренировка (fine-tuning)
model.train(
    data='data.yaml',       # путь к твоему датасету
    epochs=150,             # побольше эпох для дообучения
    imgsz=1280,             # больше размер — лучше детект людей
    batch=16,               # уменьшить, если мало памяти
    optimizer="AdamW",      # лучше для стабильности
    lr0=0.002,              # чуть меньше, чтобы не "затирать" предобученные веса
    patience=50,            # ранняя остановка, если метрика не растёт
    augment=True,           # включаем аугментации !!!!
    close_mosaic=15,        # отключаем mosaic ближе к концу
    project='runs/passengers',
    name='yolov8m-passengers',
    pretrained=True         # дообучаем предобученные веса
)


In [ ]:
# train.py
from ultralytics import YOLO
import torch

# Выбираем устройство
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется: {device}")

# Загружаем предобученную модель
model = YOLO('yolov10n.pt')  # можно yolov8s.pt для точности

# Обучение
model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    device=device,
    name='yolov10n_bus_stop_finetuned',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train'
)

# Сохранение
model.save('models/yolov10n_best.pt')
print("✅ Модель дообучена и сохранена.")

In [ ]:
# Загружаем лучшую сохранённую модель
model = YOLO("runs/passenger_flow/yolov10m_person/weights/best.pt")

# Предсказание на папке с картинками
results = model.predict(
    source="dataset/test/images",
    imgsz=640,
    conf=0.25,  # минимальная уверенность
    save=True,   # сохранить предсказания
    save_txt=True
)

In [ ]:
metrics = model.val(
    data="dataset/data.yaml",
    imgsz=640,
    conf=0.25
)

# Подсчёт пассажиров в видеопотоке

In [ ]:
from ultralytics import YOLO

model = YOLO('best.pt')

# Запуск трекинга на видео
model.track(
    source='video.mp4',
    tracker='bytetrack.yaml',  # можно 'deepsort.yaml'
    conf=0.5,
    save=True
)


In [ ]:
# упрощённый фрагмент
if hasattr(res, 'masks') and res.masks is not None:
    im_with_masks = res.masks.render(res.orig_img)  # иногда работает напрямую
    frame = im_with_masks


In [ ]:
def dice_coef(gt_mask, pred_mask):
    gt = gt_mask.astype(bool)
    pr = pred_mask.astype(bool)
    inter = (gt & pr).sum()
    return 2*inter / (gt.sum() + pr.sum() + 1e-6)
